# Building AI Agents in Pure Python
Click this [link](https://www.youtube.com/watch?v=bZzyPscbtI8&t=1116s) to follow along 
1. Chat completions
2. Structured output
3. Tool calling
4. Retrieval

In [39]:
# Load libraries and env variables
import json
import os
import requests

from dotenv import load_dotenv
from openai import AzureOpenAI
from pydantic import BaseModel, Field

load_dotenv()

True

In [40]:
# Initialize OpenAI client
client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version=os.getenv("OPENAI_API_VERSION")
)

### 1. Chat completions

In [ ]:
# Create chat completion
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You're a helpful assistant."},
        {"role": "user", "content": "Write a limerick about the Python programming language.",
        },
    ],
)

In [42]:
# Parse the response
response = completion.choices[0].message.content
print(response)

In a land where the coders all cheer,  
Python’s syntax is simple and clear.  
With libraries vast,  
And functions amassed,  
It makes programming a pleasure, my dear!  


### 2. Structured output

In [43]:
# Define the response format in a Pydantic model
class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

In [44]:
# Call the model
completion = client.beta.chat.completions.parse(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "Extract the event information."},
        {
            "role": "user",
            "content": "Alice and Bob are going to a science fair on Friday.",
        },
    ],
    response_format=CalendarEvent,
)

In [45]:
# Parse the response
event = completion.choices[0].message.parsed
print(event)

name='Science Fair' date='Friday' participants=['Alice', 'Bob']


### 3. Tool calling

In [46]:
# Define the tool (function) that we want to call - docs: https://platform.openai.com/docs/guides/function-calling
def get_weather(latitude, longitude):
    """This is a publically available API that returns the weather for a given location."""
    response = requests.get(
        f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m"
    )
    data = response.json()
    return data["current"]

In [47]:
# Call model with get_weather tool defined
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get current temperature for provided coordinates in celsius.",
            "parameters": {
                "type": "object",
                "properties": {
                    "latitude": {"type": "number"},
                    "longitude": {"type": "number"},
                },
                "required": ["latitude", "longitude"],
                "additionalProperties": False,
            },
            "strict": True,
        },
    }
]

system_prompt = "You are a helpful weather assistant."

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": "What's the weather like in Paris today?"},
]

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)

In [48]:
# Model decides to call function(s)
completion.model_dump()['choices']

[{'finish_reason': 'tool_calls',
  'index': 0,
  'logprobs': None,
  'message': {'content': None,
   'refusal': None,
   'role': 'assistant',
   'audio': None,
   'function_call': None,
   'tool_calls': [{'id': 'call_Atc3R2xGpLlshkwXxt2yJKZ5',
     'function': {'arguments': '{"latitude":48.8566,"longitude":2.3522}',
      'name': 'get_weather'},
     'type': 'function'}]},
  'content_filter_results': {}}]

In [49]:
# Execute get_weather function
def call_function(name, args):
    if name == "get_weather":
        return get_weather(**args)

for tool_call in completion.choices[0].message.tool_calls:
    name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)
    messages.append(completion.choices[0].message)

    result = call_function(name, args)
    messages.append(
        {"role": "tool", "tool_call_id": tool_call.id, "content": json.dumps(result)}
    )

In [50]:
# Supply result and call model again
class WeatherResponse(BaseModel):
    temperature: float = Field(
        description="The current temperature in celsius for the given location."
    )
    response: str = Field(
        description="A natural language response to the user's question."
    )

completion_2 = client.beta.chat.completions.parse(
    model="gpt-4o",
    messages=messages,
    tools=tools,
    response_format=WeatherResponse,
)

# Check model response
final_response = completion_2.choices[0].message.parsed
final_response.temperature
final_response.response

'The current temperature in Paris is 13.6°C with a wind speed of 6.9 m/s.'

### 4. Retrieval

In [77]:
# Define the knowledge base retrieval tool - docs: https://platform.openai.com/docs/guides/function-calling
def search_kb(question: str):
    """
    Load the whole knowledge base from the JSON file.
    (This is a mock function for demonstration purposes, we don't search)
    """
    with open("kb.json", "r") as f:
        return json.load(f)

In [78]:
# Call model with search_kb tool defined
tools = [
    {
        "type": "function",
        "function": {
            "name": "search_kb",
            "description": "Get the answer to the user's question from the knowledge base.",
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {"type": "string"},
                },
                "required": ["question"],
                "additionalProperties": False,
            },
            "strict": True,
        },
    }
]

system_prompt = "You are a helpful assistant that answers questions from the knowledge base about our e-commerce store."

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": "What is the return policy?"},
]

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)

In [79]:
# Model decides to call function(s)
completion.model_dump()['choices']

[{'finish_reason': 'tool_calls',
  'index': 0,
  'logprobs': None,
  'message': {'content': None,
   'refusal': None,
   'role': 'assistant',
   'audio': None,
   'function_call': None,
   'tool_calls': [{'id': 'call_KG0ToS9yQQV42KFwAU5AnkHn',
     'function': {'arguments': '{"question":"What is the return policy?"}',
      'name': 'search_kb'},
     'type': 'function'}]},
  'content_filter_results': {}}]

In [80]:
# Execute search_kb function
def call_function(name, args):
    if name == "search_kb":
        return search_kb(**args)


for tool_call in completion.choices[0].message.tool_calls:
    name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)
    messages.append(completion.choices[0].message)

    result = call_function(name, args)
    messages.append(
        {"role": "tool", "tool_call_id": tool_call.id, "content": json.dumps(result)}
    )

In [55]:
# Supply result and call model again
class KBResponse(BaseModel):
    answer: str = Field(description="The answer to the user's question.")
    source: int = Field(description="The record id of the answer.")


completion_2 = client.beta.chat.completions.parse(
    model="gpt-4o",
    messages=messages,
    tools=tools,
    response_format=KBResponse,
)

final_response = completion_2.choices[0].message.parsed
final_response.answer

'Items can be returned within 30 days of purchase with original receipt. Refunds will be processed to the original payment method within 5-7 business days.'

In [88]:
# Question that doesn't trigger the tool
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": "What is the weather in Tokyo?"},
]

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)

print(completion.model_dump()['choices'])

for tool_call in completion.choices[0].message.tool_calls:
    name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)
    messages.append(completion.choices[0].message)

    result = call_function(name, args)
    messages.append(
        {"role": "tool", "tool_call_id": tool_call.id, "content": json.dumps(result)}
    )
    
completion_2 = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)

completion_2.choices[0].message.content

[{'finish_reason': 'tool_calls', 'index': 0, 'logprobs': None, 'message': {'content': None, 'refusal': None, 'role': 'assistant', 'audio': None, 'function_call': None, 'tool_calls': [{'id': 'call_xvf6RTT9oJCKxoopxTZzEnKr', 'function': {'arguments': '{"question":"What is the weather in Tokyo?"}', 'name': 'search_kb'}, 'type': 'function'}]}, 'content_filter_results': {}}]


"I'm sorry, but I don't have access to real-time weather information for Tokyo or any location. You may want to check a weather website or app for the most current conditions. If you have other questions related to our e-commerce store, feel free to ask!"